In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ../..

import gymnasium as gym
import torch, numpy as np, torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import tianshou as ts

/home/ubuntu/sustaingym


In [2]:
task = 'CartPole-v0'
lr, epoch, batch_size = 1e-3, 10, 64
train_num, test_num = 10, 100
gamma, n_step, target_freq = 0.9, 3, 320
buffer_size = 20000
eps_train, eps_test = 0.1, 0.05
step_per_epoch, step_per_collect = 10000, 10
logger = ts.utils.TensorboardLogger(SummaryWriter('log/dqn'))  # TensorBoard is supported!
# For other loggers: https://tianshou.readthedocs.io/en/master/tutorials/logger.html

In [3]:
# you can also try with SubprocVectorEnv
train_envs = ts.env.DummyVectorEnv([lambda: gym.make(task) for _ in range(train_num)])
test_envs = ts.env.DummyVectorEnv([lambda: gym.make(task) for _ in range(test_num)])

/home/ubuntu/.conda/envs/sustaingymnasium/lib/python3.9/site-packages/gymnasium/envs/registration.py:578: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [4]:
from tianshou.utils.net.common import Net
# you can define other net by following the API:
# https://tianshou.readthedocs.io/en/master/tutorials/dqn.html#build-the-network
env = gym.make(task)
state_shape = env.observation_space.shape or env.observation_space.n
action_shape = env.action_space.shape or env.action_space.n
net = Net(state_shape=state_shape, action_shape=action_shape, hidden_sizes=[128, 128, 128])
optim = torch.optim.Adam(net.parameters(), lr=lr)

In [5]:
policy = ts.policy.DQNPolicy(net, optim, gamma, n_step, target_update_freq=target_freq)
train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(buffer_size, train_num), exploration_noise=True)
test_collector = ts.data.Collector(policy, test_envs, exploration_noise=True)  # because DQN uses epsilon-greedy method

In [6]:
result = ts.trainer.offpolicy_trainer(
    policy, train_collector, test_collector, epoch, step_per_epoch, step_per_collect,
    test_num, batch_size, update_per_step=1 / step_per_collect,
    train_fn=lambda epoch, env_step: policy.set_eps(eps_train),
    test_fn=lambda epoch, env_step: policy.set_eps(eps_test),
    stop_fn=lambda mean_rewards: mean_rewards >= env.spec.reward_threshold,
    logger=logger)
print(f'Finished training! Use {result["duration"]}')

Epoch #1: 10001it [00:09, 1023.06it/s, env_step=10000, len=200, loss=0.312, n/ep=0, n/st=10, rew=200.00]                          


Epoch #1: test_reward: 135.980000 ± 19.120136, best_reward: 135.980000 ± 19.120136 in #1


Epoch #2: 10001it [00:07, 1387.47it/s, env_step=20000, len=160, loss=0.350, n/ep=0, n/st=10, rew=160.00]                           


Epoch #2: test_reward: 194.110000 ± 9.254075, best_reward: 194.110000 ± 9.254075 in #2


Epoch #3:  49%|####9     | 4920/10000 [00:03<00:03, 1355.71it/s, env_step=24920, len=200, n/ep=1, n/st=10, rew=200.00]            

Finished training! Use 21.83s


In [7]:
# torch.save(policy.state_dict(), 'dqn.pth')
# policy.load_state_dict(torch.load('dqn.pth'))

In [7]:
isinstance(env, gym.Env)

True

In [10]:
isinstance(env, gym.Env) and not hasattr(env, "__len__")

True

In [12]:
from tianshou.env import BaseVectorEnv, DummyVectorEnv
test_env = DummyVectorEnv([lambda: env])

In [13]:
policy.eval()
policy.set_eps(eps_test)
collector = ts.data.Collector(policy, test_env, exploration_noise=True)
collector.collect(n_episode=1, render=1 / 35)

/home/ubuntu/.conda/envs/sustaingymnasium/lib/python3.9/site-packages/gymnasium/envs/classic_control/cartpole.py:212: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("CartPole-v0", render_mode="rgb_array")
  gym.logger.warn(


{'n/ep': 1,
 'n/st': 200,
 'rews': array([200.]),
 'lens': array([200]),
 'idxs': array([0]),
 'rew': 200.0,
 'len': 200.0,
 'rew_std': 0.0,
 'len_std': 0.0}